In [11]:
from typing_extensions import TypedDict
from typing import Literal
from langgraph.graph import StateGraph , START, END 
from IPython.display import Image, display

Define the State Class of the Graph as a TypedDict
example: calculate Agent -> States saves a number as float

In [22]:
class State(TypedDict):
    graph_state: float

Define the **Nodes** of the Graph as functions 

In [64]:
import random

def start_value(state): 
    print("--Node0: Start with:" + str(state['graph_state']) + "--")
    return {"graph_state": state['graph_state']}

def add_rn(state):
    add_rn = random.random()
    print("--Node1:Add " + str(add_rn)+ " to--")
    return {"graph_state": float(state["graph_state"]) + add_rn}

def divide_rn(state):
    div_rn = random.randrange(1, 100, 5)
    print("--Node2:Divide by " + str(div_rn) + "--")
    return {"graph_state": float(state["graph_state"]) / div_rn}

def multiply_rn(state):
    mulit_rn = random.random()
    print("--Node3:multiply by "+ str(mulit_rn) + "--")
    return {"graph_state": float(state["graph_state"]) * mulit_rn}

Define the **edges** of the Graph!

In [29]:
def decide_operation(state) -> Literal["add_rn","divide_rn","multiply_rn"]:
    if random.randrange(0,100) < 33:
        return "add_rn"
    elif random.randrange(34,100) < 66:
        return "divide_rn"
    else: return "multiply_rn"

Build the **Graph** with the builder that inherit form StateGraph

In [65]:
builder = StateGraph(State) #the builder is an Instanze of the StateGraph Class that inherits from State

#Add the nodes
builder.add_node("start_value", start_value)
builder.add_node("add_rn", add_rn)
builder.add_node("divide_rn", divide_rn)
builder.add_node("multiply_rn", multiply_rn)

#adds the edges
builder.add_edge(START, "start_value")
builder.add_conditional_edges("start_value", decide_operation)
builder.add_edge("add_rn",END)
builder.add_edge("divide_rn", END)
builder.add_edge("multiply_rn", END)

#compile the Graph
graph = builder.compile()

#Inspect the Graph
#display(Image(graph.get_graph().draw_mermaid_png()))

**invoke** the Graph with an intial value of the State. The inital vlaue is a dict with k:graph_state and v: init vlaue

In [72]:
graph.invoke({"graph_state": 3.989977})

--Node0: Start with:3.989977--
--Node2:Divide by 11--


{'graph_state': 0.3627251818181818}

In [31]:
graph.invoke({"graph_state": 3.1})

--Node0: Start--
--Node1:Add--


{'graph_state': 3.482986295377531}

## defining the message state and reducers

In [2]:
from typing_extensions import TypedDict
from langchain_core.messages import AnyMessage

class MessagesState(TypedDict):
    messages : list[AnyMessage]

By default the messsges get override after each node runs. To prevent this we need to add the "add_message" reducer and Annotde the value of messages with the reducer

In [3]:
from typing import Annotated
from langgraph.graph.message import add_messages

class MessagesState(TypedDict):
    messages :  Annotated[list[AnyMessage], add_messages]

or use a prebuild MessageState from langgraph.graph

In [ ]:
from langgraph.graph import MessagesState

class MessagesState(MessagesState):
    pass

### Build the graph using MessagesState as our State

In [10]:
from IPython.display import  Image, display
from langgraph.graph import START, END, MessagesState, StateGraph

class MessagesState(MessagesState):
    pass

def tool_calling_llm(state: MessagesState):
    return{"messages": [llm_with_tools.invoke(state["messages"])]}

